In [1]:
# Merged files 
import os
import pandas as pd
import numpy as np

In [3]:
dirpath = os.getcwd()
features_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir),"udder_processing",  "udder_features")
dfv1 = pd.read_csv(os.path.join(features_dir, "volumes.csv"), index_col=0)
dfr1 = pd.read_csv(os.path.join(features_dir, "region_props.csv"), index_col=0)
dfr1["lp_group"] = "l"
dfv1["lp_group"] = "l"
dfv2 = pd.read_csv(os.path.join(features_dir, "volumes_newcows.csv"), index_col=0)
dfr2 = pd.read_csv(os.path.join(features_dir, "region_props_newcows.csv"), index_col=0)
dfr2["lp_group"] = "p"
dfv2["lp_group"] = "p"

dfr = pd.concat([dfr1, dfr2], axis = 0, ignore_index = True)
dfv = pd.concat([dfv1, dfv2], axis = 0, ignore_index = True)

df_all = dfr.merge(dfv, on=["cow", "filename", "lp_group"])

id_vars = ["cow", "lp_group"]
features = ["udder_ecc", "lf_ecc", "rf_ecc", "lb_ecc", "rb_ecc", "volume", "lf_vol", "rf_vol", "lb_vol", "rb_vol"]
df_all2= pd.melt(df_all, id_vars = id_vars, value_vars = features).reset_index()
grouped = df_all2[["cow","lp_group" ,"variable", "value"]].groupby(["cow", "lp_group", "variable"]).agg([np.nanmedian, np.nanmin,np.nanmax,np.nanmean, "count"])
grouped.columns = [b.replace("nan", "") if b else a for a, b in grouped.columns]
grouped = grouped.reset_index()

grouped_wide = grouped[["cow", "lp_group", "variable", "median"]].pivot(index = id_vars, columns = "variable", values = "median").reset_index()

grouped_wide.to_csv("udder_features_median.csv", index = False)
df_all.to_csv("udder_features_raw.csv", index = False)
grouped.to_csv("udder_features_grouped.csv", index = False)